# 🚀 **Módulo 2: Arquitetura MCP - O Plano do Metrô da IA**

## **Aula 2.1: Como MCP funciona por baixo dos panos**

---

### **Tá, mas como MCP funciona na prática?**

Imagine que você está em São Paulo e quer ir do Shopping Morumbi até o Shopping Cidade São Paulo. Você tem duas opções:

**Opção 1 - Sem metrô**: Você precisa conhecer cada rua, cada semáforo, cada esquina. Se uma rua estiver fechada, você se perde completamente.

**Opção 2 - Com metrô**: Você só precisa saber que existe uma linha que conecta os dois pontos. O metrô se encarrega de todo o resto!

**MCP é o metrô da IA!** É um **protocolo padronizado** que permite que IAs e ferramentas se comuniquem sem precisar conhecer os detalhes um do outro.

**Por que isso é revolucionário?**

Antes do MCP, cada IA precisava **programar individualmente** cada ferramenta. Era como ter que aprender a dirigir em cada cidade. Com MCP, existe um **padrão universal** - é como ter uma carteira de motorista que vale em qualquer lugar!

---

**🖼️ Sugestão de imagem**: Um mapa de metrô conectando diferentes estações (IAs e ferramentas)

### **Setup Inicial - Preparando o Terreno**

Vamos configurar nosso ambiente para entender a arquitetura MCP. É como preparar os instrumentos para dissecar um sistema!

In [ ]:
# 🆓 SETUP GRATUITO PARA COLAB
# Instalando as dependências necessárias
!pip install -q langchain langchain-community langchain-core python-dotenv
!pip install -q huggingface_hub langchain-openai openai
!pip install -q requests json5

print("✅ Dependências instaladas com sucesso!")

**🎯 O que acabamos de instalar?**

As mesmas ferramentas do módulo anterior, mas agora vamos **entender como elas funcionam por baixo dos panos**. É como ter as peças de um motor e agora vamos ver como elas se encaixam!

Agora vamos configurar as **chaves de acesso** e entender a **arquitetura MCP**.

In [ ]:
# 🔑 CONFIGURAÇÃO DE API KEYS
import os
from dotenv import load_dotenv

# Carregando variáveis de ambiente (se existirem)
load_dotenv()

def get_llm_colab():
    """Retorna o melhor LLM disponível no Colab"""
    
    # Tentativa 1: OpenAI (se você tiver dinheiro)
    try:
        from langchain_openai import ChatOpenAI
        api_key = os.getenv("OPENAI_API_KEY")
        if api_key:
            print("�� Usando OpenAI GPT-3.5-turbo")
            return ChatOpenAI(
                model="gpt-3.5-turbo",
                temperature=0.7,
                api_key=api_key
            )
    except Exception as e:
        print(f"❌ OpenAI não disponível: {e}")
    
    # Tentativa 2: Hugging Face (GRATUITO!)
    try:
        from langchain_community.llms import HuggingFaceHub
        token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
        if token:
            print("🎉 Usando Hugging Face (GRATUITO!)")
            return HuggingFaceHub(
                repo_id="google/flan-t5-base",
                model_kwargs={"temperature": 0.7, "max_length": 512}
            )
    except Exception as e:
        print(f"❌ Hugging Face não disponível: {e}")
    
    # Fallback: Simulador simples
    print("⚠️ Usando simulador local (sem API keys)")
    return None

# Configurando o LLM
llm = get_llm_colab()

if llm:
    print("✅ LLM configurado com sucesso!")
else:
    print("⚠️ Executando em modo simulação (sem LLM real)")

**🔑 Configuração pronta!**

Agora vamos **dissecar** a arquitetura MCP. É como abrir um relógio para ver como as engrenagens funcionam!

**💡 Lembrete**: Se você não configurou as API keys, não tem problema! Vamos entender os conceitos mesmo assim.

## **Aula 2.2: Protocolo, mensagens e comunicação**

---

### **O protocolo MCP - As regras do jogo**

MCP funciona como um **protocolo de comunicação** entre IAs e ferramentas. É como ter um **idioma universal** que todos entendem.

**Analogia**: Imagine que você está num **aeroporto internacional**:

- **Pessoas de diferentes países** = IAs e ferramentas
- **Inglês** = Protocolo MCP
- **Conversas** = Mensagens MCP
- **Aeroporto** = Ambiente MCP

**Como funciona na prática?**

1. **IA** quer fazer algo ("Quero ler um arquivo")
2. **IA** envia mensagem MCP para o servidor
3. **Servidor MCP** processa e responde
4. **IA** recebe o resultado

Vamos ver isso **código por código**!

In [ ]:
# ��️ ARQUITETURA MCP - ENTENDENDO O PROTOCOLO

import json
from typing import Dict, Any, List
from dataclasses import dataclass
from datetime import datetime

@dataclass
class MensagemMCP:
    """Representa uma mensagem MCP - como um envelope com instruções"""
    tipo: str  # Tipo da mensagem (tool_call, tool_result, etc.)
    id: str    # ID único da mensagem
    dados: Dict[str, Any]  # Conteúdo da mensagem
    timestamp: str  # Quando foi enviada
    
    def to_json(self) -> str:
        """Converte a mensagem para JSON (formato MCP)"""
        return json.dumps({
            "type": self.tipo,
            "id": self.id,
            "data": self.dados,
            "timestamp": self.timestamp
        }, indent=2)

class ServidorMCP:
    """Simula um servidor MCP básico"""
    
    def __init__(self, nome: str):
        self.nome = nome
        self.ferramentas = {}
        self.historico = []
        
    def registrar_ferramenta(self, nome: str, funcao):
        """Registra uma ferramenta no servidor"""
        self.ferramentas[nome] = funcao
        print(f"🔧 Ferramenta '{nome}' registrada no servidor {self.nome}")
        
    def processar_mensagem(self, mensagem: MensagemMCP) -> MensagemMCP:
        """Processa uma mensagem MCP e retorna a resposta"""
        
        print(f"📨 Servidor {self.nome} recebeu: {mensagem.tipo}")
        self.historico.append(mensagem)
        
        if mensagem.tipo == "tool_call":
            # IA quer usar uma ferramenta
            ferramenta = mensagem.dados.get("tool")
            parametros = mensagem.dados.get("params", {})
            
            if ferramenta in self.ferramentas:
                print(f"🔧 Executando ferramenta: {ferramenta}")
                resultado = self.ferramentas[ferramenta](**parametros)
                
                # Criando resposta
                resposta = MensagemMCP(
                    tipo="tool_result",
                    id=f"resp_{mensagem.id}",
                    dados={"result": resultado, "tool": ferramenta},
                    timestamp=datetime.now().isoformat()
                )
                
                print(f"✅ Resultado enviado: {resultado}")
                return resposta
            else:
                # Ferramenta não encontrada
                erro = MensagemMCP(
                    tipo="error",
                    id=f"erro_{mensagem.id}",
                    dados={"error": f"Ferramenta '{ferramenta}' não encontrada"},
                    timestamp=datetime.now().isoformat()
                )
                return erro
        
        # Mensagem não reconhecida
        return MensagemMCP(
            tipo="error",
            id=f"erro_{mensagem.id}",
            dados={"error": f"Tipo de mensagem '{mensagem.tipo}' não suportado"},
timestamp=datetime.now().isoformat()
        )

# Criando nosso servidor MCP
servidor = ServidorMCP("Servidor de Arquivos")

print(f"🏗️ Servidor MCP '{servidor.nome}' criado!")
print(f"�� Ferramentas disponíveis: {list(servidor.ferramentas.keys())}")

**🎯 O que acabamos de criar?**

Criamos a **estrutura básica** de um servidor MCP:

- ✅ **MensagemMCP**: Como um "envelope" com instruções
- ✅ **ServidorMCP**: O "aeroporto" onde as conversas acontecem
- ✅ **Registro de ferramentas**: Como "lojas" no aeroporto
- ✅ **Processamento de mensagens**: Como "atendentes" que entendem pedidos

**�� Analogia**: É como criar um **shopping center** onde:
- Cada loja é uma ferramenta
- Os atendentes entendem um idioma universal
- Os clientes (IAs) podem pedir qualquer coisa

Agora vamos **adicionar ferramentas** ao nosso servidor!

In [ ]:
# 🔧 ADICIONANDO FERRAMENTAS AO SERVIDOR MCP

# Ferramenta 1: Ler arquivo
def ler_arquivo(caminho: str) -> str:
    """Lê um arquivo e retorna o conteúdo"""
    try:
        with open(caminho, 'r', encoding='utf-8') as arquivo:
            conteudo = arquivo.read()
            return f"Arquivo '{caminho}' lido com sucesso! Conteúdo: {conteudo[:100]}..."
    except FileNotFoundError:
        return f"Erro: Arquivo '{caminho}' não encontrado"
    except Exception as e:
        return f"Erro ao ler arquivo: {e}"

# Ferramenta 2: Listar arquivos
def listar_arquivos(diretorio: str = ".") -> str:
    """Lista arquivos em um diretório"""
    try:
        import os
        arquivos = os.listdir(diretorio)
        return f"Arquivos em '{diretorio}': {', '.join(arquivos[:10])}"
    except Exception as e:
        return f"Erro ao listar arquivos: {e}"

# Ferramenta 3: Criar arquivo
def criar_arquivo(caminho: str, conteudo: str) -> str:
    """Cria um arquivo com conteúdo"""
    try:
        with open(caminho, 'w', encoding='utf-8') as arquivo:
            arquivo.write(conteudo)
        return f"Arquivo '{caminho}' criado com sucesso!"
    except Exception as e:
        return f"Erro ao criar arquivo: {e}"

# Registrando as ferramentas no servidor
servidor.registrar_ferramenta("ler_arquivo", ler_arquivo)
servidor.registrar_ferramenta("listar_arquivos", listar_arquivos)
servidor.registrar_ferramenta("criar_arquivo", criar_arquivo)

print(f"\n�� Ferramentas disponíveis: {list(servidor.ferramentas.keys())}")
print("✅ Servidor MCP pronto para receber pedidos!")

**�� Ferramentas registradas!**

Agora nosso servidor MCP tem **3 ferramentas especializadas**:

- �� **ler_arquivo**: Lê conteúdo de arquivos
- 📋 **listar_arquivos**: Lista arquivos em diretórios
- ✏️ **criar_arquivo**: Cria novos arquivos

**�� Analogia**: É como ter um **escritório inteligente** com:
- Um **arquivista** que lê documentos
- Um **organizador** que lista o que tem
- Um **secretário** que cria novos documentos

Agora vamos **testar a comunicação** entre IA e servidor MCP!

In [ ]:
# 🧪 TESTANDO A COMUNICAÇÃO MCP

# Simulando uma IA que quer usar o servidor MCP
def ia_usando_mcp(pedido: str):
    """Simula uma IA que usa o servidor MCP"""
    
    print(f"🤖 IA: {pedido}")
    print("-" * 50)
    
    # IA analisa o pedido e decide qual ferramenta usar
    if "ler" in pedido.lower() and "arquivo" in pedido.lower():
        # IA quer ler um arquivo
        ferramenta = "ler_arquivo"
        # Extraindo o caminho do arquivo (simulação simples)
        if "README" in pedido:
            caminho = "README.md"
        else:
            caminho = "arquivo_teste.txt"
        parametros = {"caminho": caminho}
        
    elif "listar" in pedido.lower() or "quais" in pedido.lower():
        # IA quer listar arquivos
        ferramenta = "listar_arquivos"
        parametros = {"diretorio": "."}
        
    elif "criar" in pedido.lower() or "novo" in pedido.lower():
        # IA quer criar um arquivo
        ferramenta = "criar_arquivo"
        parametros = {
            "caminho": "arquivo_criado.txt",
            "conteudo": "Este arquivo foi criado pela IA usando MCP!"
        }
        
    else:
        print("🤖 IA: Desculpe, não tenho uma ferramenta para esse pedido.")
        return
    
    # Criando mensagem MCP
    mensagem = MensagemMCP(
        tipo="tool_call",
        id=f"pedido_{datetime.now().timestamp()}",
        dados={"tool": ferramenta, "params": parametros},
        timestamp=datetime.now().isoformat()
    )
    
    print(f"📨 IA enviando mensagem MCP: {mensagem.tipo}")
    print(f"�� Ferramenta solicitada: {ferramenta}")
    print(f"�� Parâmetros: {parametros}")
    
    # Servidor processa a mensagem
    resposta = servidor.processar_mensagem(mensagem)
    
    print(f"\n�� Servidor responde: {resposta.tipo}")
    print(f"✅ Resultado: {resposta.dados.get('result', resposta.dados.get('error'))}")
    
    print("-" * 50)

# Testando diferentes pedidos
print("🧪 TESTANDO COMUNICAÇÃO IA + MCP")
print("=" * 60)

pedidos_teste = [
    "Quero ler o arquivo README",
    "Quais arquivos existem neste diretório?",
    "Cria um novo arquivo para mim",
    "Lê o arquivo que acabei de criar"
]

for pedido in pedidos_teste:
    ia_usando_mcp(pedido)
    print()

print("🎉 Comunicação MCP funcionando perfeitamente!")

**🎯 UAU! O que acabamos de ver?**

Uma **comunicação completa** entre IA e servidor MCP:

1. **IA** faz um pedido em linguagem natural
2. **IA** analisa e decide qual ferramenta usar
3. **IA** envia mensagem MCP para o servidor
4. **Servidor** processa e executa a ferramenta
5. **Servidor** retorna o resultado
6. **IA** recebe e apresenta o resultado

**💡 O que isso prova?**

Que MCP é **realmente um protocolo universal**:

- ✅ **Padronizado**: Todas as mensagens seguem o mesmo formato
- ✅ **Flexível**: Aceita qualquer tipo de ferramenta
- ✅ **Inteligente**: IA pode escolher a ferramenta certa
- ✅ **Confiável**: Sempre retorna uma resposta estruturada

**🚀 Agora vamos ver o "plano do metrô" completo!**

In [ ]:
# 🗺️ O PLANO DO METRÔ MCP - ARQUITETURA COMPLETA

import json
from typing import Dict, List, Any

class ArquiteturaMCP:
    """Representa a arquitetura completa do MCP"""
    
    def __init__(self):
        self.estacoes = {}  # Servidores MCP
        self.linhas = {}    # Conexões entre servidores
        self.ia_central = None
        
    def adicionar_estacao(self, nome: str, servidor: ServidorMCP):
        """Adiciona uma estação (servidor) ao metrô MCP"""
        self.estacoes[nome] = servidor
        print(f"🚇 Estação '{nome}' adicionada ao metrô MCP")
        
    def conectar_estacoes(self, estacao1: str, estacao2: str):
        """Conecta duas estações (servidores podem se comunicar)"""
        if estacao1 not in self.linhas:
            self.linhas[estacao1] = []
        if estacao2 not in self.linhas:
            self.linhas[estacao2] = []
            
        self.linhas[estacao1].append(estacao2)
        self.linhas[estacao2].append(estacao1)
        print(f"�� Linha criada: {estacao1} ↔ {estacao2}")
        
    def rotear_pedido(self, pedido: str) -> Dict[str, Any]:
        """Roteia um pedido para o servidor correto"""
        
        print(f"🗺️ Roteando pedido: '{pedido}'")
        
        # Análise simples para decidir qual servidor usar
        if any(palavra in pedido.lower() for palavra in ["arquivo", "ler", "criar", "listar"]):
            servidor_alvo = "arquivos"
        elif any(palavra in pedido.lower() for palavra in ["calcular", "soma", "multiplica"]):
            servidor_alvo = "calculadora"
        elif any(palavra in pedido.lower() for palavra in ["banco", "dados", "salvar"]):
            servidor_alvo = "banco_dados"
        else:
            servidor_alvo = "geral"
            
        print(f"�� Servidor escolhido: {servidor_alvo}")
        
        if servidor_alvo in self.estacoes:
            return {
                "servidor": servidor_alvo,
                "disponivel": True,
                "ferramentas": list(self.estacoes[servidor_alvo].ferramentas.keys())
            }
        else:
            return {
                "servidor": servidor_alvo,
                "disponivel": False,
                "erro": f"Servidor '{servidor_alvo}' não encontrado"
            }
    
    def mostrar_mapa(self):
        """Mostra o mapa completo do metrô MCP"""
        print("\n��️ MAPA DO METRÔ MCP")
        print("=" * 50)
        
        for estacao, servidor in self.estacoes.items():
            print(f"\n🚇 ESTAÇÃO: {estacao}")
            print(f"   📋 Servidor: {servidor.nome}")
            print(f"   �� Ferramentas: {list(servidor.ferramentas.keys())}")
            
            if estacao in self.linhas:
                conexoes = self.linhas[estacao]
                print(f"   �� Conectado com: {', '.join(conexoes)}")
        
        print("\n🎯 IA Central pode acessar todas as estações!")

# Criando nossa arquitetura MCP
metro_mcp = ArquiteturaMCP()

# Adicionando estações (servidores)
servidor_arquivos = ServidorMCP("Servidor de Arquivos")
servidor_arquivos.registrar_ferramenta("ler_arquivo", ler_arquivo)
servidor_arquivos.registrar_ferramenta("listar_arquivos", listar_arquivos)
servidor_arquivos.registrar_ferramenta("criar_arquivo", criar_arquivo)

servidor_calc = ServidorMCP("Servidor de Cálculos")
servidor_calc.registrar_ferramenta("somar", lambda a, b: a + b)
servidor_calc.registrar_ferramenta("multiplicar", lambda a, b: a * b)

metro_mcp.adicionar_estacao("arquivos", servidor_arquivos)
metro_mcp.adicionar_estacao("calculadora", servidor_calc)

# Conectando as estações
metro_mcp.conectar_estacoes("arquivos", "calculadora")

# Mostrando o mapa
metro_mcp.mostrar_mapa()

**🎯 IMPRESSIONANTE! O que acabamos de criar?**

Uma **arquitetura MCP completa** com:

- 🚇 **Múltiplas estações** (servidores especializados)
- 🔗 **Linhas de conexão** (servidores podem se comunicar)
- ��️ **Sistema de roteamento** (IA escolhe o servidor certo)
- �� **IA central** que coordena tudo

**�� Analogia**: É como ter um **metrô inteligente** onde:

- Cada estação é um **serviço especializado**
- As linhas são **conexões entre serviços**
- O roteamento é **inteligente**
- A IA é o **passageiro** que sabe para onde quer ir

**�� Agora vamos testar o roteamento!**

In [ ]:
# 🧪 TESTANDO O ROTEAMENTO MCP

def testar_roteamento_mcp(pedidos: List[str]):
    """Testa o roteamento de diferentes pedidos"""
    
    print("�� TESTANDO ROTEAMENTO MCP")
    print("=" * 60)
    
    for i, pedido in enumerate(pedidos, 1):
        print(f"\n📋 TESTE {i}: '{pedido}'")
        print("-" * 40)
        
        # Roteando o pedido
        rota = metro_mcp.rotear_pedido(pedido)
        
        print(f"�� Servidor escolhido: {rota['servidor']}")
        print(f"✅ Disponível: {rota['disponivel']}")
        
        if rota['disponivel']:
            print(f"�� Ferramentas disponíveis: {rota['ferramentas']}")
            
            # Simulando execução
            servidor = metro_mcp.estacoes[rota['servidor']]
            print(f"⚡ Executando no servidor: {servidor.nome}")
        else:
            print(f"❌ Erro: {rota['erro']}")
        
        print("-" * 40)

# Testando diferentes tipos de pedidos
pedidos_teste = [
    "Quero ler o arquivo README",
    "Calcula 15 + 27",
    "Lista os arquivos do diretório",
    "Multiplica 5 por 8",
    "Cria um novo arquivo",
    "Me conta uma piada"  # Este não tem servidor específico
]

testar_roteamento_mcp(pedidos_teste)

print("\n🎉 Roteamento MCP funcionando perfeitamente!")

## **Teste Rápido**

---

Vamos testar se você entendeu a arquitetura MCP!

### **Pergunta 1**:
Qual é a principal vantagem da arquitetura MCP?

**A)** É mais barata
**B)** Permite que IAs se comuniquem com ferramentas usando um protocolo universal
**C)** É mais rápida
**D)** É mais bonita

### **Pergunta 2**:
Como funciona o roteamento em MCP?

**A)** A IA sempre usa o primeiro servidor disponível
**B)** A IA analisa o pedido e escolhe o servidor mais adequado
**C)** O servidor escolhe qual IA usar
**D)** É aleatório

**�� Respostas**: 1-B, 2-B

Se você acertou, parabéns! Você entende a arquitetura MCP!

## **Desafio do Módulo**

---

### **Crie um novo servidor MCP!**

Vamos criar um **servidor de banco de dados** que pode:

- **Salvar dados** ("salva o nome João com idade 25")
- **Buscar dados** ("busca todos os usuários com idade 25")
- **Atualizar dados** ("atualiza a idade do João para 26")
- **Deletar dados** ("remove o usuário João")

**Desafio**: Implemente essas 4 ferramentas e adicione o servidor ao metrô MCP!

In [ ]:
# 🎯 DESAFIO: Servidor de Banco de Dados MCP

# TODO: Implemente o servidor de banco de dados
# Dica: Use um dicionário Python como "banco de dados" simples

class ServidorBancoDados:
    """Servidor MCP para operações de banco de dados"""
    
    def __init__(self):
        self.nome = "Servidor de Banco de Dados"
        self.dados = {}  # Nosso "banco de dados" simples
        self.ferramentas = {}
        
    def registrar_ferramenta(self, nome: str, funcao):
        """Registra uma ferramenta no servidor"""
        self.ferramentas[nome] = funcao
        
    # TODO: Implemente as 4 ferramentas:
    # 1. salvar_dados
    # 2. buscar_dados  
    # 3. atualizar_dados
    # 4. deletar_dados

# Criando e testando o servidor
servidor_bd = ServidorBancoDados()

# TODO: Registre as ferramentas e teste!

print("�� DESAFIO: Implemente o servidor de banco de dados!")

## **Resumo do Módulo 2**

---

### **O que aprendemos hoje:**

✅ **MCP funciona como um metrô** - protocolo universal de comunicação
✅ **Mensagens MCP** - formato padronizado para troca de informações
✅ **Servidores MCP** - estações especializadas com ferramentas
✅ **Roteamento inteligente** - IA escolhe o servidor correto
✅ **Arquitetura escalável** - fácil adicionar novos servidores

### **Conceitos-chave:**

- **Protocolo MCP** = Regras universais de comunicação
- **Mensagem MCP** = Envelope com instruções padronizadas
- **Servidor MCP** = Estação com ferramentas especializadas
- **Roteamento** = Sistema que escolhe o servidor certo
- **Arquitetura** = Plano do metrô completo

### **Próximos passos:**

No próximo módulo, vamos explorar **Tools, Resources e Prompts** - as ferramentas específicas que tornam MCP tão poderoso. É como entender o **"arsenal de ferramentas"** da IA!

---

**�� Dica do Pedro**: A arquitetura MCP é como um **ecossistema inteligente**. Cada servidor é especializado, mas todos falam a mesma língua. É a diferença entre ter um **exército desorganizado** e um **exército bem treinado**!

**🚀 Próximo módulo**: Ferramentas MCP - O Arsenal da IA